# Problema de las N-Reinas 2.0

Explicación del problema: https://youtu.be/ZGKSoTUaphU


Explicación del código: https://youtu.be/3wVwpPzoyDs


Nuestra función a minimizar será:


$\begin{equation}
f(x) =  -\sum_i^n \sum_j^n x_{i,j} +   \lambda\sum_{i_1}^n \sum_{i_2}^n \sum_{i_3}^n \sum_{i_4}^n J_{i_1,i_2,i_3,i_4}x_{i_1,i_2}x_{i_3,i_4}
\end{equation}$

donde $x_{ij}$ valdrá 1 si en la casilla (i,j) hay una dama y 0 si no.

Daremos además a $J_{i_1,i_2,i_3,i_4}$ el valor 1 si las casillas $(i_1,i_2)$ y $(i_3, i_4)$ están conectadas y 0 en otro caso.

In [1]:
import qubovert

size = 6
lagrange = size ** 2

# Creamos las variables de nuestro modelo
Q = qubovert.QUBO()
for i in range(size):
    for j in range(size):
        Q.create_var(f"x_{i}_{j}")
        
# Añadimos el primer bloque de la función objetivo
for i in range(size):
    for j in range(size):
        Q[(f"x_{i}_{j}",)] = -1
        
# Incluimos las restricciones finales
for i1 in range(size):
    for i2 in range(size):
        for i3 in range(size):
            for i4 in range(size):
                if i1 == i3 or i2 == i4 or i1 - i3 == i2 - i4 or i1 - i3 == i4 - i2:
                    if not (i1 == i3 and i2 == i4):
                        Q[(f"x_{i1}_{i2}", f"x_{i3}_{i4}")] = lagrange
                

In [4]:
dwave_dic = {}
for i in Q:
    if len(i) == 1:
        dwave_dic[(i[0],i[0])] = Q[i]
    else:
        dwave_dic[i] = Q[i]

In [5]:
from neal import SimulatedAnnealingSampler
#from dwave.system import DWaveSampler, EmbeddingComposite


n_samples = 2000 # número de veces que ejecutamos el sistema


sampler = SimulatedAnnealingSampler()
#sampler = EmbeddingComposite(DWaveSampler())

sampleset = sampler.sample_qubo(dwave_dic, num_reads = n_samples)
solution = sampleset.first.sample

In [6]:
for i in range(size):
    for j in range(size):
        if solution[f"x_{i}_{j}"] == 0:
            print("O", end = " ")
        else:
            print("X", end = " ")
    print()

O O O X O O 
X O O O O O 
O O O O X O 
O X O O O O 
O O O O O X 
O O X O O O 
